In [1]:
def intersect(df1, df2, columns):
    df1 = df1.drop_duplicates(columns)
    df2 = df2.drop_duplicates(columns)
    merged = df1.append(df2)
    
    return merged[merged.duplicated(columns)]

In [2]:
def minus(df2, df1, columns):
    df2 = df2[columns]
    df1 = df1[columns]
    return pd.concat([df2, df1, df1]).drop_duplicates(keep=False)

In [3]:
import pandas as pd

In this notebook we will try to answer to some questions about the challange dataset, the questions are:

- Are Holdings in Challange?
- Do they ask us some interactions never happened?
- When did they start sampling the negative interactions?
- How the negative samples in challange have been chosen? uniformly at random?


Let's start by answering to the first quesiton. We will try to find some touples (CustomerIdx, IsinIdx, BuySell) that are in trades with trade status = 'Holding' but are not in trade with trade status != "Holding". If none of these touplets are present in challange then 'Holdings' are not in challenge.

Let's start by importing the dataframes:

In [4]:
trade = pd.read_csv("../data/Trade.csv")
chall = pd.read_csv("../data/Challenge_20180423.csv")

let's take just the interactions form the starting of 2018

In [5]:
trade = trade[trade["TradeDateKey"] > 20180000]
trade = trade.rename(index=str, columns={"TradeDateKey": "DateKey"})
trade_holding = trade[trade["TradeStatus"] == "Holding"]
trade_interest = trade[trade["TradeStatus"] != "Holding"]

In [6]:
trade_holding.head()

,DateKey,CustomerIdx,IsinIdx,BuySell,NotionalEUR,Price,TradeStatus,CustomerInterest
2212067,20180131,989,20305,Sell,34428.0,NaN,Holding,0.0
2212076,20180131,989,21534,Sell,590192.0,NaN,Holding,0.0
2212122,20180131,989,21535,Buy,7904724.0,NaN,Holding,0.0
2212123,20180228,989,21535,Sell,7904724.0,NaN,Holding,0.0
2212208,20180131,989,23023,Buy,1068175.0,NaN,Holding,0.0


In [7]:
print(trade_holding.shape)
print(trade_interest.shape)

(498989, 8)
(350121, 8)


Let's now filter for the unique values of the tuples CustomerIdx, IsinIdx and BuySell

In [8]:
trade_holding = trade_holding.drop_duplicates(["CustomerIdx", "IsinIdx", "BuySell"])
trade_interest = trade_interest.drop_duplicates(["CustomerIdx", "IsinIdx", "BuySell"])

In [9]:
print(trade_holding.shape)
print(trade_interest.shape)

(366703, 8)
(208426, 8)


let's now take the holdings not in interest

In [10]:
unique_holdings = minus(trade_holding,trade_interest,["CustomerIdx", "IsinIdx", "BuySell"])

In [11]:
unique_holdings.shape

(329982, 3)

Let's see now how many of them are in challenge

In [12]:
holding_in_chall = intersect(unique_holdings, chall, ["CustomerIdx", "IsinIdx", "BuySell"])

print("the holdings not in interest are {}".format(unique_holdings.shape[0]))
print("the holdings in challange but not in trade_interest are {}".format(holding_in_chall.shape[0]))

the holdings not in interest are 329982
the holdings in challange but not in trade_interest are 26318


In [13]:
trade_interest_in_chall = intersect(trade_interest, chall, ["CustomerIdx", "IsinIdx", "BuySell"])

print("the trade interest are {}".format(trade_interest.shape[0]))
print("the trade interest in challange are {}".format(trade_interest_in_chall.shape[0]))

the trade interest are 208426
the trade interest in challange are 195963


As you can see, almost all the trade interests are in challange, but just the 10% of unique holding are in challange and the percentage of a holdings with the same ["CustomerIdx", "IsinIdx", "BuySell"] of an interest is also around 10%. So, we can assume that that at the beginning there was one big dataset trades. This dataframe has been used to generate the test dataset. Then this dataset has been splitted in two halves and we have just one half. The unique holdings that are in challange are actually trades from the second half that by coincidence have the same ["CustomerIdx", "IsinIdx", "BuySell"] of our holdings. If holdings were included in challange the percentage of unique holdings in challange would be much higher. So 

## Holdings are not in the test dataset

# Second Quesiton

Let's now see how many interactions in challange already happened in the past:

In [14]:
trade = pd.read_csv("../data/Trade.csv")
chall_already_happened = intersect(trade, chall, ["CustomerIdx", "IsinIdx", "BuySell"])
print("{} out of {} interactions in challange already happened".format(chall_already_happened.shape[0],chall.shape[0]))

360140 out of 484758 interactions in challange already happened


I assume that the other interactions come from the second half of trades that we don't have

# Third Question

to answer to this question we will look to some transactions that happened in the past and never happened again in the future. If none of these interactions is in challange then that past has not been used to generate the negative examples 

In [15]:
trade = trade.rename(index=str, columns={"TradeDateKey": "DateKey"})
trade = trade[trade["TradeStatus"] != "Holding"]
past = trade[trade["DateKey"] < 20170000].drop_duplicates(["CustomerIdx", "IsinIdx", "BuySell"])
future = trade[trade["DateKey"] > 20170000].drop_duplicates(["CustomerIdx", "IsinIdx", "BuySell"])

past_not_in_future = minus(past, future,["CustomerIdx", "IsinIdx", "BuySell"])
past_not_in_future.shape

(309870, 3)

In [16]:
u_p_in_chall = intersect(past_not_in_future, chall_already_happened,["CustomerIdx", "IsinIdx", "BuySell"])
u_p_in_chall.shape

(17355, 10)

Since some interactions are present just in the 2017 and are present in the challange generated from our samples we can assume that all the transations starting from the 01-01-2016 has been used to generate the negative samples in the test set so 
## All of them has been used

# Fourth quesition

Let's now see if they have been chosen uniformly or not.

In [17]:
trades_2016 = trade[trade["DateKey"] < 20170000].drop_duplicates(["CustomerIdx", "IsinIdx", "BuySell"])
trades_2017 = trade[trade["DateKey"] > 20170000]
trades_2017 = trades_2017[trades_2017["DateKey"] < 20180000].drop_duplicates(["CustomerIdx", "IsinIdx", "BuySell"])
trades_2018 = trade[trade["DateKey"] > 20180000].drop_duplicates(["CustomerIdx", "IsinIdx", "BuySell"])

only_2016 = minus(trades_2016, trades_2017,["CustomerIdx", "IsinIdx", "BuySell"])
only_2016 = minus(only_2016, trades_2018,["CustomerIdx", "IsinIdx", "BuySell"])

only_2017 = minus(trades_2017, trades_2016,["CustomerIdx", "IsinIdx", "BuySell"])
only_2017 = minus(only_2017, trades_2018, ["CustomerIdx", "IsinIdx", "BuySell"])

only_2018 = minus(trades_2018, trades_2016,["CustomerIdx", "IsinIdx", "BuySell"])
only_2018 = minus(only_2018, trades_2017, ["CustomerIdx", "IsinIdx", "BuySell"])

chall_2016 = intersect(only_2016, chall_already_happened,["CustomerIdx", "IsinIdx", "BuySell"])
chall_2017 = intersect(only_2017, chall_already_happened,["CustomerIdx", "IsinIdx", "BuySell"])
chall_2018 = intersect(only_2018, chall_already_happened,["CustomerIdx", "IsinIdx", "BuySell"])

print("{} out of {} happened only in 2016 are in challange".format(chall_2016.shape[0], only_2016.shape[0]))
print("{}%".format(round(100*chall_2016.shape[0]/only_2016.shape[0])))
print("{} out of {} happened only in 2017 are in challange".format(chall_2017.shape[0], only_2017.shape[0]))
print("{}%".format(round(100*chall_2017.shape[0]/only_2017.shape[0])))
print("{} out of {} happened only in 2018 are in challange".format(chall_2018.shape[0], only_2018.shape[0]))
print("{}%".format(round(100*chall_2018.shape[0]/only_2018.shape[0])))

17355 out of 309870 happened only in 2016 are in challange
6%
88528 out of 323803 happened only in 2017 are in challange
27%
117724 out of 124550 happened only in 2018 are in challange
95%


It's quite clear to see that the more recent the transaction is the more likely it has been picked. 
Let's now breakdown the transactions by months before. We will see 6, 8 and 10 months before 2018-04-22



In [18]:
trades_six_months = trade[trade["DateKey"] > 20171021].drop_duplicates(["CustomerIdx", "IsinIdx", "BuySell"])
others = trade[trade["DateKey"] <= 20171021].drop_duplicates(["CustomerIdx", "IsinIdx", "BuySell"])

only_six_months = minus(trades_six_months, others,["CustomerIdx", "IsinIdx", "BuySell"])

chall_six_months = intersect(only_six_months, chall_already_happened,["CustomerIdx", "IsinIdx", "BuySell"])

print("{} out of {} happened only in the last six months are in challange".format(chall_six_months.shape[0], only_six_months.shape[0]))
print("{}%".format(round(100*chall_six_months.shape[0]/only_six_months.shape[0])))

177634 out of 189366 happened only in the last six months are in challange
94%


In [19]:
trades_eight_months = trade[trade["DateKey"] > 20170821].drop_duplicates(["CustomerIdx", "IsinIdx", "BuySell"])
others = trade[trade["DateKey"] <= 20170821].drop_duplicates(["CustomerIdx", "IsinIdx", "BuySell"])

only_eight_months = minus(trades_eight_months, others,["CustomerIdx", "IsinIdx", "BuySell"])

chall_eight_months = intersect(only_eight_months, chall_already_happened,["CustomerIdx", "IsinIdx", "BuySell"])

print("{} out of {} happened only in the last eight months challange".format(chall_eight_months.shape[0], only_eight_months.shape[0]))
print("{}%".format(round(100*chall_eight_months.shape[0]/only_eight_months.shape[0])))

193905 out of 246173 happened only in the last eight months challange
79%


In [20]:
trades_ten_months = trade[trade["DateKey"] > 20170621].drop_duplicates(["CustomerIdx", "IsinIdx", "BuySell"])
others = trade[trade["DateKey"] <= 20170621].drop_duplicates(["CustomerIdx", "IsinIdx", "BuySell"])

only_ten_months = minus(trades_ten_months, others,["CustomerIdx", "IsinIdx", "BuySell"])

chall_ten_months = intersect(only_ten_months, chall_already_happened,["CustomerIdx", "IsinIdx", "BuySell"])

print("{} out of {} happened only in the last 10 months are in challange".format(chall_ten_months.shape[0], only_ten_months.shape[0]))
print("{}%".format(round(100*chall_ten_months.shape[0]/only_ten_months.shape[0])))

207824 out of 304985 happened only in the last 10 months are in challange
68%


It's clear that the more months we add the less probability we have and that the transactions happened 6 months before and 4 months before (the one of 2018) have both the same probability of being in challange (95%)

In [21]:
older_six_months = trade[trade["DateKey"] <= 20170621].drop_duplicates(["CustomerIdx", "IsinIdx", "BuySell"])
older_six_months = older_six_months[older_six_months["DateKey"] > 20160621].drop_duplicates(["CustomerIdx", "IsinIdx", "BuySell"])
even_older = trade[trade["DateKey"] <= 20160621].drop_duplicates(["CustomerIdx", "IsinIdx", "BuySell"])

only_older_six_months = minus(older_six_months, trades_six_months,["CustomerIdx", "IsinIdx", "BuySell"])
only_older_six_months_one_year = minus(only_older_six_months, even_older,["CustomerIdx", "IsinIdx", "BuySell"])


chall_older_six_months = intersect(only_older_six_months, chall_already_happened,["CustomerIdx", "IsinIdx", "BuySell"])
chall_older_six_months_one_year = intersect(only_older_six_months_one_year, chall_already_happened,["CustomerIdx", "IsinIdx", "BuySell"])

print("{} out of {} happened only more than six months before are in challange".format(chall_older_six_months.shape[0], only_older_six_months.shape[0]))
print("{}%".format(round(100*chall_older_six_months.shape[0]/only_older_six_months.shape[0])))
print("{} out of {} happened only between 18 and 6 months before are in challange".format(chall_older_six_months_one_year.shape[0], only_older_six_months_one_year.shape[0]))
print("{}%".format(round(100*chall_older_six_months_one_year.shape[0]/only_older_six_months_one_year.shape[0])))

33195 out of 371695 happened only more than six months before are in challange
9%
28706 out of 342335 happened only between 18 and 6 months before are in challange
8%


Transcations older than 6 months have the same probability of being chosen, the one in 2016 is 6%, between 18 and 6 months 8% and older than 6 months 9% 

How many transactions happened in the last 6 monts happened also before ? 

In [22]:
intersection = intersect(trades_six_months, older_six_months,["CustomerIdx", "IsinIdx", "BuySell"])

print("{} out of {} happened at the most six months before already happened before".format(intersection.shape[0], six_months_in_chall.shape[0]))
print("{}%".format(round(100*intersection.shape[0]/six_months_in_chall.shape[0])))

NameError: name 'six_months_in_chall' is not defined

# Conclusions
The negative samples in Challange were created by taking the 95% of unique interactions from 2017-10-22 and the 8% of the unique interactions happened before. Since not all the rows in challange are in trades, it is reasonable to think that the dataset used to generate challange is bigger than the one we have. One can think that the 8% of interactions happened before are actually transactions happened after the 2017-10-22 but from the bigger dataset that by coincidence have th same ["CustomerIdx", "IsinIdx", "BuySell"] of an older transaction. However in our dataset this coincidence is 24% and since 24 >> (8 / 0.95),  that's not the case.

And Holding were probably just removed.

In [25]:
chall.groupby(["CustomerIdx", "IsinIdx"]).size().reset_index(name='counts').describe()

,CustomerIdx,IsinIdx,counts
count,242379.000000,242379.000000,242379.0
mean,1936.835749,15572.977102,2.0
std,869.366108,8381.699897,0.0
min,0.000000,1.000000,2.0
25%,1288.000000,7950.000000,2.0
50%,2106.000000,15592.000000,2.0
75%,2574.000000,23841.000000,2.0
max,3470.000000,27344.000000,2.0


In [27]:
chall_already_happened = intersect(trade, chall, ["CustomerIdx", "IsinIdx"])
print("{} out of {} interactions in challange already happened".format(chall_already_happened.shape[0],chall.drop_duplicates(["CustomerIdx", "IsinIdx"]).shape[0]))

242379 out of 242379 interactions in challange already happened


In [31]:
chall_already_happened = intersect(trade[trade["DateKey"] > 20171022], chall, ["CustomerIdx", "IsinIdx"])
print("{} out of {} interactions in challange already happened".format(chall_already_happened.shape[0],chall.drop_duplicates(["CustomerIdx", "IsinIdx"]).shape[0]))

242379 out of 242379 interactions in challange already happened


In [33]:
trade[trade["DateKey"] > 20171022].drop_duplicates(["CustomerIdx", "IsinIdx"]).shape[0]

260382

In [37]:
trade[trade["DateKey"] >= 20171022].drop_duplicates(["CustomerIdx","IsinIdx"]).shape[0]

260382